In [2]:
data_file = './dataloader/data/KGS-2002-19-3646-test-v2.dat'

def get_board_dimensions(data_file):
    f = open(data_file, "rb")
    try:
        header_bytes = f.read(1024)
        comps = header_bytes.split('-')
        if comps[0] != 'mlv2':
            raise Exception('Not a properly processed go data file')
        n = int(comps[1].split('=')[1])
        num_planes = int(comps[2].split('=')[1])
        image_width = int(comps[3].split('=')[1])
        image_height = int(comps[4].split('=')[1])
        if image_width != image_height:
            raise Exception('Board size and width do not match.')
        else:
            board_size = image_width
    finally:
        f.close()
    return n, num_planes, board_size

print get_board_dimensions(data_file)

(628, 7, 19)


In [5]:
def get_record_size(num_planes, board_size):
    board_squared = board_size ** 2
    record_size = 6 # two for the initial word 'GO' and four for the label
    num_bits = num_planes * board_squared
    num_bytes = ( num_bits + 8 - 1 ) / 8
    record_size += num_bytes;
    return board_squared, record_size

In [13]:
def load_go_data(data_file, start_record = 0, num_records = None):
    n, num_planes, board_size = get_board_dimensions(data_file)
    if num_records is None:
        num_records = n - start_record
    board_squared, record_size = get_record_size(num_planes, board_size);
    pos = start_record * record_size + 1024 # account for header
    chunk_byte_size = num_records * record_size
    
    labels = []
    
    f = open(data_file, "rb")
    try:
        pre_start_bytes = f.read(pos) # Throw these away
        for batch in range(num_records):
            data = f.read(record_size)
            if str(data[:2]) != 'GO':
                raise Exception('Invalid alignment of go games')
            label = data[2]
            labels += [ord(label)]
            
            bit_position = 0
            intra_record_position = 0
            planes = data[6:]
            this_record_byte = planes[0]
            
            
            for plane in range(num_planes):
                start = (batch * num_planes + plane) * board_squared
                data_plane = planes[start:]
                #print data_plane
                for position in range(board_squared):
                    this_byte = ( ord(this_record_byte) >> ( 7 - bit_position ) ) & 1
                    #if position == 0:
                    #print data_plane[position]
                    bit_position += 1
                    if bit_position == 8:
                        bit_position = 0
                        intra_record_position += 1
                        this_record_byte = planes[intra_record_position]
    finally:
        f.close()
    print(labels)
    return None

load_go_data(data_file)

                                                                                                                                                                                                                                         @                             @                                                    
                                                                                                                                                                                                                                                                                                                         
                                                                                                                                                                                                                                        @                             @�                                                   
                                                 